In [1]:
from train_graph_davis import *

/home/jyjiang/ls/envs/drug/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-04-11 14:47:30,243] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/jyjiang/ls/envs/drug/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/jyjiang/ls/envs/drug/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/jyjiang/ls/envs/drug/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/jyjiang/ls/envs/drug/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/jyjiang/ls/envs/drug/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/jyjiang/ls/envs/drug/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/jyjiang/ls/envs/drug/compiler_compat/ld: /usr/local/cuda/lib64

In [2]:
davis_drug_path = '/home/jyjiang/drug_interaction/data/drug.csv'
davis_target_path = '/home/jyjiang/drug_interaction/data/bind_data/target.csv'
davis_drug_feature_path = '/home/jyjiang/drug_interaction/data/bind_data/kpgt_base.npz'
davis_protein_feature_path = '/home/jyjiang/drug_interaction/data/bind_data/prot_rep.pkl'
davis_data_path = '/home/jyjiang/drug_interaction/data/bind_data/data.csv'
davis_graph_folderpath = '/home/jyjiang/drug_interaction/data/bind_data/graph'

In [3]:
davis_drug_feature = np.load(davis_drug_feature_path)
davis_drug_feature = davis_drug_feature['fps']
with  open(davis_protein_feature_path, 'rb') as f:
    davis_protein_feature = pickle.load(f)
# average to get same length
davis_target_feature = []
for feature in davis_protein_feature:
    feature = feature.squeeze(0)
    davis_target_feature.append(feature[1 : len(feature[0]) - 1].mean(0))
# build dict of id and feature     
davis_drug_df = pd.read_csv(davis_drug_path)
davis_target_df = pd.read_csv(davis_target_path)
davis_drug_ids = davis_drug_df['Drug_ID']
davis_target_ids = davis_target_df['Target_ID']
davis_drug_dict = dict(zip(davis_drug_ids, davis_drug_feature))
# davis_target_dict = dict(zip(davis_target_ids, davis_target_feature))
davis_data_df=pd.read_csv(davis_data_path)
davis_data_df['Label']=davis_data_df['Label'].astype(int)
# davis_drug_id_to_index = {drug_id: idx for idx, drug_id in enumerate(davis_drug_df['Drug_ID'])}
# davis_target_id_to_index = {target_id: idx for idx, target_id in enumerate(davis_target_df['Target_ID'])}

In [4]:
test_dataset = GraphDataset_withsim(davis_data_df, davis_drug_df, davis_target_df, davis_drug_dict, davis_graph_folderpath)
valid_dataset = GraphDataset_withsim(davis_data_df, davis_drug_df, davis_target_df, davis_drug_dict, davis_graph_folderpath)
weight = torch.tensor([1.0, 2.0]).to(torch.device(f'cuda:0'))
criterion = DTIContrastiveLoss()
criterion2 = nn.CrossEntropyLoss(weight=weight)
sim_loss_fn = SimilarityLoss()

In [5]:
batch_size_cam=1

In [52]:
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size_cam,collate_fn=custom_collate_fn_zero,shuffle=False)

In [8]:
class GraphDTI(nn.Module):
    def __init__(self, drug_dim, prot_dim, output_dim, perturb=False, surface_feature=False):
        super(GraphDTI, self).__init__()
        self.esm_linear = nn.Linear(prot_dim, 512)
        self.esm_linear_sf = nn.Linear(prot_dim, 448)
        # self.gcn = torch.nn.DataParallel(GraphCNN(pooling='MTP'), device_ids=[0, 1])
        #self.gcn = GraphCNN(pooling='MTP')
        self.gcn = GraphCNN(pooling='MTP')
        self.surface_linear = nn.Linear(3, 16)
        self.surface_linear2 = nn.Linear(16, 64)
        
        self.drug_linear = nn.Linear(drug_dim, 1024)
        self.hidden2 = nn.Linear(1024, 256)
        self.drug_prot_linear = nn.Linear(512, 256)
        self.outlinear = nn.Linear(256, output_dim)
        self.perturb = perturb
        self.surface_feature = surface_feature
        self.gradients = None
    def activations_hook(self, grad):
        self.gradients = grad
    def get_features_gradients(self):
        return self.gradients
    def forward(self, drug, prot_data):
        if self.surface_feature:
            x_esm = self.esm_linear_sf(prot_data.x)
            x_sf = self.surface_linear(prot_data.surface_feature)
            x_sf = self.surface_linear2(x_sf)
            x1 = torch.cat((x_esm, x_sf), dim=1)
        else:
            x1 = self.esm_linear(prot_data.x)
            x1.register_hook(self.activations_hook)
        gcn_n_feat1, gcn_g_feat1 = self.gcn(x1, prot_data)
        if self.perturb:
            gcn_n_feat2, gcn_g_feat2 = self.gcn(x1, prot_data, pertubed=True)
            
            x2 = self.drug_linear(drug)
            x2 = F.relu(x2)
            x2 = F.dropout(x2, 0.2)
            x2 = self.hidden2(x2)
            # x2 = F.relu(x2)
            # x2 = F.dropout(x2, 0.2)
            
            x3 = torch.cat((x2, gcn_g_feat1), dim=1)
            x3 = self.drug_prot_linear(x3)
            x3 = F.relu(x3)
            x3 = F.dropout(x3, 0.2)
            out = self.outlinear(x3)
            
            return out, gcn_g_feat1, gcn_g_feat2
        
        else:
            x2 = self.drug_linear(drug)
            x2 = F.relu(x2)
            x2 = F.dropout(x2, 0.2)
            x2 = self.hidden2(x2)
            # x2 = F.relu(x2)
            # x2 = F.dropout(x2, 0.2)
            
            x3 = torch.cat((x2, gcn_g_feat1), dim=1)
            x3 = self.drug_prot_linear(x3)
            x3 = F.relu(x3)
            x3 = F.dropout(x3, 0.2)
            out = self.outlinear(x3)
            return out,x2, gcn_g_feat1
            #return out, x2, gcn_g_feat1

In [21]:
from Bio import PDB
from collections import defaultdict

def extract_residue_numbers(pdb_file):
    """Extract all residue numbers from a PDB file"""
    parser = PDB.PDBParser(QUIET=True)
    structure = parser.get_structure('protein', pdb_file)
    
    L=[]
    for model in structure:
        for chain in model:
            residue_numbers = set()
            for residue in chain:
                if PDB.is_aa(residue):  
                    resnum = residue.get_id()[1]
                    residue_numbers.add(resnum)
            L.append(sorted(residue_numbers))
    return L
    

In [23]:
def get_bindpos(full_pdb,partial_pdb):
    partial_residues = extract_residue_numbers(partial_pdb)
    full_residues=extract_residue_numbers(full_pdb)
    positions=[]
    prev=0
    for i in range(len(partial_residues)):
        if partial_residues[i]:
            positions+=[(res+prev) for res in partial_residues[i]] 
            prev+=max(full_residues[i])
    return positions

In [60]:
from Bio.PDB import PDBParser, is_aa
from Bio.SeqUtils import seq1

def extract_sequence(pdbfile):
    """
    Extract the sequence of amino acids from a PDB file using Biopython's PDBParser.
    """
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure(pdbfile.split('/')[-1].split('.')[0], pdbfile)
    
    # Extract residues and convert to a sequence
    residues = [residue for residue in structure.get_residues() if is_aa(residue, standard=True)]
    sequence = "".join([seq1(residue.get_resname()) for residue in residues])  # Convert to one-letter code
    
    return sequence

In [63]:
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
def get_bindpos(full_pdb, partial_pdb):
    """
    Compute the binding positions of the partial PDB file in the full PDB file
    using sequence alignment.
    """
    # Extract sequences from the PDB files
    full_sequence = extract_sequence(full_pdb)
    partial_sequence = extract_sequence(partial_pdb)
    
    # Perform sequence alignment
    alignments = pairwise2.align.globalxx(full_sequence, partial_sequence)

    # Use the best alignment (highest score)
    best_alignment = alignments[0]
    full_aligned, partial_aligned, score, start, end = best_alignment

    # Map positions from the aligned sequences
    positions = []
    full_index = 0
    partial_index = 0

    for full_res, partial_res in zip(full_aligned, partial_aligned):
        if full_res == partial_res and partial_res != "-":
            positions.append(full_index + 1)  # Add 1 to convert to 1-based indexing
        if full_res != "-":
            full_index += 1
        if partial_res != "-":
            partial_index += 1

    return positions

/home/jyjiang/ls/envs/drug/lib/python3.12/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [29]:
L=davis_data_df['Target_ID'].tolist()

In [64]:
bindposmap=[]
for pdb_id in L:
    full_pdb=f'/home/jyjiang/drug_interaction/data/demo/{pdb_id}/{pdb_id}_protein.pdb'
    partial_pdb=f'/home/jyjiang/drug_interaction/data/demo/{pdb_id}/{pdb_id}_pocket.pdb'
    bindposmap.append( get_bindpos(full_pdb,partial_pdb))

In [199]:
model = GraphDTI(bindingdb_drug_feature[0].shape[0], bindingdb_target_feature[0].shape[0], 2).to(torch.device(f'cuda:0'))
model.load_state_dict(torch.load('/home/jyjiang/drug_interaction/best_model79.pth'))
model.eval()

/tmp/ipykernel_3807893/4036695014.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/home/jyjiang/drug_interaction/best_model79.pth'))


GraphDTI(
  (esm_linear): Linear(in_features=1280, out_features=512, bias=True)
  (esm_linear_sf): Linear(in_features=1280, out_features=448, bias=True)
  (gcn): GraphCNN(
    (gcn): ModuleList(
      (0-2): 3 x GCNConv(512, 512)
    )
    (pool): GraphMultisetTransformer(512, 256, pool_sequences=['GMPool_G', 'GMPool_G'])
    (drop1): Dropout(p=0.2, inplace=False)
  )
  (surface_linear): Linear(in_features=3, out_features=16, bias=True)
  (surface_linear2): Linear(in_features=16, out_features=64, bias=True)
  (drug_linear): Linear(in_features=2304, out_features=1024, bias=True)
  (hidden2): Linear(in_features=1024, out_features=256, bias=True)
  (drug_prot_linear): Linear(in_features=512, out_features=256, bias=True)
  (outlinear): Linear(in_features=256, out_features=2, bias=True)
)

In [53]:
len(test_loader)

252

In [ ]:
matchness.append(np.sum( [1 if heatmap[x-1]>0 else 0 for x in bindposmap[i]])/len(bindposmap[i]))

In [187]:
matchness=[]
maps=[]
for i,b in enumerate(test_loader):
    outputs,emb1, emb2 = model(b[0],b[1])
    loss = criterion2(outputs.float(), b[2])
    with torch.no_grad():
      loss.backward()
    pooled_gradients = torch.mean(model.gradients.squeeze(), dim=[1])
    heatmap=pooled_gradients.cpu()
    heatmap=np.maximum(0, heatmap)
    maps.append(heatmap)
    matchness.append(np.mean([heatmap[x-1] for x in bindposmap[i]])/torch.mean(heatmap) )

In [172]:
import numpy as np

def check_zero_percentage(tensor, per=0.3):
    """
    Check if the percentage of zeros in the tensor is less than or equal to `per`.
    """
    # Count the number of zeros in the tensor
    zero_count = (tensor == 0).sum().item()
    total_elements = tensor.numel()
    
    # Calculate the percentage of zeros
    zero_percentage = zero_count / total_elements
    
    # Return whether the zero percentage is within the specified limit
    return zero_percentage <= per

def top_n_indices(lst, maps, n, per=0.3):
    """
    Get the indices of the top N maximum values in `lst` filtered by zero percentage in `maps`.
    
    Parameters:
    - lst: List of values (e.g., scores).
    - maps: List of tensors corresponding to `lst`.
    - n: Number of top indices to return.
    - per: Maximum allowed percentage of zeros in the tensors in `maps`.
    """
    # Convert the list to a NumPy array
    arr = np.array(lst)
    
    # Filter out NaN values from the array
    valid_indices = np.argwhere(~np.isnan(arr)).flatten()
    valid_values = arr[valid_indices]
    
    # Filter indices based on the zero percentage in `maps`
    filtered_indices = [
        idx for idx in valid_indices 
        if check_zero_percentage(maps[idx], per=per)
    ]
    
    # Sort the filtered indices by the values in `lst` (descending order)
    sorted_indices = sorted(
        filtered_indices, key=lambda idx: arr[idx], reverse=True
    )
    
    # Return the top N indices (or fewer if there aren't enough valid indices)
    return sorted_indices[:n]

In [189]:
top_n_indices(matchness, maps,5)

[81, 146, 17, 222, 229]

In [190]:
[L[i] for i in top_n_indices(matchness,maps, 5)]

['1z6e', '1lpg', '4ddh', '1p1n', '2yki']

In [191]:
select=[maps[i] for i in top_n_indices(matchness, maps,5)]

In [192]:
select[0]

tensor([8.8711e-05, 1.0924e-04, 8.6138e-05, 5.8467e-05, 3.5810e-05, 2.5157e-05,
        2.1785e-05, 1.6240e-05, 1.3509e-05, 1.6405e-05, 1.1433e-05, 2.3069e-05,
        2.2455e-05, 3.6293e-05, 4.3669e-05, 4.1387e-05, 3.4197e-05, 2.5251e-05,
        1.7393e-05, 1.3475e-05, 2.4970e-06, 0.0000e+00, 0.0000e+00, 3.3077e-06,
        1.2921e-05, 3.0715e-05, 3.7822e-05, 4.2307e-05, 4.2585e-05, 4.4579e-05,
        3.3327e-05, 3.0045e-05, 1.4901e-05, 1.2059e-05, 1.0929e-05, 2.1361e-05,
        2.4150e-05, 3.8419e-05, 4.3813e-05, 5.1407e-05, 3.7158e-05, 5.0221e-05,
        4.0603e-05, 3.9604e-05, 4.1038e-05, 3.9646e-05, 3.2175e-05, 3.2851e-05,
        2.3494e-05, 3.2634e-05, 3.5833e-05, 3.8479e-05, 4.2803e-05, 3.9381e-05,
        3.2314e-05, 1.0178e-05, 2.3944e-07, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        3.9319e-05, 4.0357e-05, 3.6364e-05, 3.3645e-05, 3.9074e-05, 3.2557e-05,
        3.7088e-05, 3.1398e-05, 2.9715e-

In [193]:
torch.save(select, "select_heatmap.pt")

In [200]:
evaluate(model, test_loader, criterion2,device='cuda')

(6.555453642593658,
 21.03174603174603,
 0,
 0.21031746031746032,
 1.0,
 0.21031746031746032,
 0.3475409836065574,
 0,
 0,
 0)

In [10]:
%cd data

/home/jyjiang/drug_interaction/data


/home/jyjiang/ls/envs/drug/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
import os
import csv
from rdkit import Chem
from rdkit.Chem import AllChem
from Bio import SeqIO
from Bio.PDB.PDBParser import PDBParser
import numpy as np
# Paths
base_folder = "/home/jyjiang/drug_interaction/data/demo"  # Replace with your actual folder path
output_data_csv = "data.csv"
output_drug_csv = "drug.csv"
output_target_csv = "target.csv"

# Initialize containers for data
data_rows = []
drug_rows = {}
target_rows = {}

# Function to extract SMILES from SDF or MOL2
def extract_smiles(file_path):
    if file_path.endswith(".sdf"):
        supplier = Chem.SDMolSupplier(file_path)
        for mol in supplier:
            if mol is not None:
                return Chem.MolToSmiles(mol)
    elif file_path.endswith(".mol2"):
        from openbabel import pybel
        mols = pybel.readfile("mol2", file_path)
        for mol in mols:
            return mol.write("smi").strip()
    return None

# Function to extract protein sequence from PDB
def load_predicted_PDB3(pdbfile):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("protein", pdbfile)
    
    sequence = []
    ca_coords = []

    # 遍历结构中的链
    for model in structure:
        for chain in model:
            for residue in chain:
                # 跳过非标准氨基酸或溶剂分子
                if residue.id[0] != " ":
                    continue
                # 提取氨基酸序列
                if "CA" in residue:
                    sequence.append(residue.resname)  # 三字母代码
                    ca_coords.append(residue["CA"].coord)  # Cα 原子坐标

    # 将三字母氨基酸代码转换为单字母代码
    aa_map = {
        'ALA': 'A', 'CYS': 'C', 'ASP': 'D', 'GLU': 'E', 'PHE': 'F',
        'GLY': 'G', 'HIS': 'H', 'ILE': 'I', 'LYS': 'K', 'LEU': 'L',
        'MET': 'M', 'ASN': 'N', 'PRO': 'P', 'GLN': 'Q', 'ARG': 'R',
        'SER': 'S', 'THR': 'T', 'VAL': 'V', 'TRP': 'W', 'TYR': 'Y', 'SEC': 'U'
    }
    sequence = ''.join([aa_map.get(res, 'X') for res in sequence])  # 转换为单字母
    
    
    num_atoms = np.array(ca_coords).shape[0]
    distance_matrix = np.zeros((num_atoms, num_atoms))
    
    for i in range(num_atoms):
        for j in range(i, num_atoms):
            distance = np.linalg.norm(ca_coords[i] - ca_coords[j])  # 欧几里得距离
            distance_matrix[i, j] = distance
            distance_matrix[j, i] = distance  # 矩阵对称

    return distance_matrix, sequence

# Traverse the demo folder
for subdir, _, files in os.walk(base_folder):
    protein_id = os.path.basename(subdir)
    ligand_sdf = os.path.join(subdir, f"{protein_id}_ligand.sdf")
    ligand_mol2 = os.path.join(subdir, f"{protein_id}_ligand.mol2")
    protein_pdb = os.path.join(subdir, f"{protein_id}_protein.pdb")

    # Skip if required files are missing
    if not os.path.exists(protein_pdb) or (not os.path.exists(ligand_sdf) and not os.path.exists(ligand_mol2)):
        continue

    # Extract SMILES
    smiles = extract_smiles(ligand_sdf if os.path.exists(ligand_sdf) else ligand_mol2)
    if not smiles:
        continue

    # Extract protein sequence
    distance_matrix,protein_seq = load_predicted_PDB3(protein_pdb)
    if not protein_seq or len(protein_seq>1200):
        continue

    # Create unique IDs for the drug and target
    drug_id = f"{len(drug_rows) + 1}"
    target_id = protein_id

    # Add to drug and target rows
    if smiles not in drug_rows:
        drug_rows[smiles] = drug_id
    if target_id not in target_rows:
        target_rows[target_id] = protein_seq

    # Add to data rows
    data_rows.append({
        "Drug_ID": drug_rows[smiles],
        "Drug": smiles,
        "Target_ID": target_id,
        "Target": protein_seq,
        "Label": 1  # Default label (can be adjusted based on your needs)
    })

# Write data.csv
with open(output_data_csv, "w", newline="") as csvfile:
    fieldnames = ["Drug_ID", "Drug", "Target_ID", "Target", "Label"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(data_rows)

# Write drug.csv
with open(output_drug_csv, "w", newline="") as csvfile:
    fieldnames = ["Drug_ID", "smiles"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for smiles, drug_id in drug_rows.items():
        writer.writerow({"Drug_ID": drug_id, "smiles": smiles})

# Write target.csv
with open(output_target_csv, "w", newline="") as csvfile:
    fieldnames = ["Target_ID", "Target"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for target_id, protein_seq in target_rows.items():
        writer.writerow({"Target_ID": target_id, "Target": protein_seq})

print("CSV files created successfully!")

[11:26:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[11:26:21] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.


[11:26:21] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[11:26:21] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[11:26:21] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[11:26:21] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[11:26:21] Explicit valence for atom # 19 C, 5, is greater than permitted
[11:26:21] ERROR: Could not sanitize molecule ending on line 78
[11:26:21] ERROR: Explicit valence for atom # 19 C, 5, is greater than permitted
[11:26:21] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[11:26:21] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[11:26:23] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3

CSV files created successfully!
